# Importações

In [ ]:
import os
import pandas as pd

# Ambiente de Execução: Colab X VSCode

In [ ]:
# # Usar no Google Colab
# data_bronze = '/content/'

In [ ]:
# Usar no VSCode
data_bronze = os.path.abspath('../data/bronze')

# Leitura e informações de apenas um arquivo
- Separado em 'chunks' devido o volume de dados

In [ ]:
def read_csv():
    f = 'mta_1706.csv'
    return pd.read_csv(os.path.join(data_bronze, f), on_bad_lines='skip', chunksize=10000) # Devido a limitação de memória, o arquivo CSV é lido em pedaços (chunks), já desconsiderando as linhas com erros de quantidade de colunas

for i, chunk in enumerate(read_csv()): # Exibir o cabeçalho do primeiro pedaço (chunk) do DataFrame
    if i == 0:
        display(chunk.head(10))
    else:
        break

In [ ]:
for i, chunk in enumerate(read_csv()): # Informações gerais do primeiro pedaço (chunk) do DataFrame
    if i == 0:
        print('Informações de nomes de colunas, tipos de dados, quantidade de valores não nulos')
        display(chunk.info())

        print('\n\nInformações estatísticas do DataFrame como: contagem, média, desvio padrão, mínimo, máximo, etc.')
        display(chunk.describe())

        print('\n\n Quantidade vallores nulls e NaN')
        display(chunk.isnull().sum())
        display(chunk.isna().sum())

        print('\n\n Quantidade de valores únicos por coluna')
        display(chunk.nunique())
    else:
        break

# Explorar o comportamento de um veículo em uma linha e no ponto de parada
- Filtrar 'VehicleRef' == 'NYCT_7571'

In [ ]:
df_veic_list = [] # Lista para guardar todos os chunks filtrados por veículo

for chunk in read_csv(): # Filtra e adiciona cada chunk na lista
    df_veic_temp = chunk[(chunk['VehicleRef'] == 'NYCT_7571')]
    df_veic_list.append(df_veic_temp)

In [ ]:
df_veic_list_concat = pd.concat(df_veic_list, ignore_index=True) # Junta todos os chunks filtrados num único DataFrame, resetando os index
display(df_veic_list_concat)

## Filtrar a coluna 'ArrivalProximityText' para 'at stop' do veiculo especifico filtrado

---



In [ ]:
display(df_veic_list_concat['ArrivalProximityText'].drop_duplicates().to_list()) # Lista todas as categorias sobre a proximidade do ponto de parada

In [ ]:
df_veic_arrival_proximity_text = df_veic_list_concat[df_veic_list_concat['ArrivalProximityText'] == 'at stop'].reset_index() # Filtra apenas informações de chegada no ponto para o veiculo especifico
display(df_veic_arrival_proximity_text)

## Filtrar a coluna 'PublishedLineName' para 'B8' do veiculo especifico filtrado, após filtro de 'at stop' para 'ArrivalProximityText'

In [ ]:
display(df_veic_arrival_proximity_text['PublishedLineName'].drop_duplicates().to_list()) # Lista todas as linhas que o veiculo filtrado operou

In [ ]:
df_veic_published_line_name = df_veic_arrival_proximity_text[df_veic_list_concat['PublishedLineName'] == 'B8'].reset_index() # Filtra apenas a linha B8 depois do filtrar o veiculo e ponto de parada
display(df_veic_published_line_name)

# Explorar os horários inicial e final dos veículos de uma linha
- Filtrar 'PublishedLineName' == 'B8'

In [ ]:
df_line_list = [] # Lista para guardar todos os chunks filtrados por veículo

for chunk in read_csv(): # Filtra e adiciona cada chunk na lista
    df_line_temp = chunk[(chunk['PublishedLineName'] == 'B8')]
    df_line_list.append(df_line_temp)

In [ ]:
df_line_list_concat = pd.concat(df_line_list, ignore_index=True) # Junta todos os chunks filtrados num único DataFrame, resetando os index
display(df_line_list_concat)

## Agrupa os dados por veiculo e data para a linha especificada
- Converte a coluna de timestamp em datetime
- Cria a coluna com a data a partir do timestamp

In [ ]:
df_line = df_line_list_concat.copy() # Copia o DataFrame
df_line['RecordedAtTime'] = pd.to_datetime(df_line['RecordedAtTime']) # Converte em tipo datetime

df_line['RecordedAtDate'] = pd.to_datetime(df_line['RecordedAtTime']).dt.date # Cria uma coluna com apenas 'data'
df_line['RecordedAtDate'] = pd.to_datetime(df_line['RecordedAtDate']) # Converte em tipo datetime

display(df_line.info())

In [ ]:
df_line_grouped = df_line.groupby(['RecordedAtDate', 'VehicleRef'])['RecordedAtTime'].agg(['min', 'max']).reset_index() # Agrupa os dados por Veiculo e Data

df_line_time_range = df_line_grouped.copy() # Copia o DataFrame
df_line_time_range['time_range'] = df_line_grouped['max'] - df_line_grouped['min'] # Adiciona uma coluna de duração da operação do Veiculo por Data

display(df_line_time_range.info())
display(df_line_time_range)

# Explorar a origem e destino de cada linha

In [ ]:
df_list_orig_dest = [] # Lista para guardar todas as rotas (origem e destino por linha)

for chunk in read_csv(): # Filtra, remove duplicatas e adiciona cada chunk na lista
    df_list_orig_dest_temp = chunk[['PublishedLineName', 'OriginName', 'DestinationName', 'DirectionRef']].drop_duplicates()
    df_list_orig_dest.append(df_list_orig_dest_temp)

In [ ]:
df_list_orig_dest_concat = pd.concat(df_list_orig_dest, ignore_index=True) # Junta todos os chunks filtrados num único DataFrame, resetando os index
df_list_orig_dest_concat.drop_duplicates(inplace=True) # { O parametro "inplace" altera o DataFrame sem necessidade de atribuir a ele mesmo }

df_list_orig_dest_concat.sort_values(by=['PublishedLineName', 'OriginName','DestinationName'], ascending=True, inplace=True, ignore_index=True) # Ordena a lista de rotas por Linha, Origem e Destino

display(df_list_orig_dest_concat)

## Verificar se o sentido 0 ou 1 são rotas diferentes ou a mesma com sentido diferente
### Exemplo de Análise:
- - "0 : 1 AV/125 ST -> INWOOD 220 ST via AMSTERDAM via BWAY"
- - "1 : INWOOD 220 ST via AMSTERDAM via BWAY -> 1 AV/125 ST"

### Separar as rotas por sentido
- - Renomear a origem e destino para TP/TS para o sentido 0, e TS/TP para o sentido 1, para verificar se as rotas são as mesmas, apenas mudando o sentido
- - - A representação do Destino no sentido 0 será o TP
- - - A representação da Origem no sentido 1 também será o TP

In [ ]:
df_route_0 = df_list_orig_dest_concat[df_list_orig_dest_concat['DirectionRef'] == 0].reset_index(drop=True) # Filtrar rotas no sentido 0

df_route_0.rename(columns={'OriginName': 'TP', 'DestinationName': 'TS'}, inplace=True) # Padronizar Origem e Destino como ponto padrão (TP/TS)

display(df_route_0)

In [ ]:
df_route_1 = df_list_orig_dest_concat[df_list_orig_dest_concat['DirectionRef'] == 1].reset_index(drop=True) # Filtrar rotas no sentido 1

df_route_1.rename(columns={'OriginName': 'TS', 'DestinationName': 'TP'}, inplace=True) # Padronizar Origem e Destino como ponto padrão (TP/TS)

display(df_route_1)

In [ ]:
# Verificar se após a separação do sentidos a quantidade de dados permaneceu
df_original = len(df_list_orig_dest_concat)
df_split_routes = len(df_route_0) + len(df_route_1)

display(df_original == df_split_routes)

## Juntar os DF de cada sentido e remover duplicadas para verificar se a quantidade mudou em relação ao original

In [ ]:
df_route = pd.concat([df_route_0, df_route_1], ignore_index=True)

df_route.drop_duplicates().reset_index(drop=True, inplace=True)

display(df_route)

- _Independente se a linha é mesma, e as origens e destinos unidos por ponto (TP/TS), a quantidade não mudou, dando a entender que as rotas são exclusivas para cada linha_

In [ ]:
# Verificar se após a união do sentidos a quantidade de dados permaneceu
display(len(df_list_orig_dest_concat) == len(df_route))